### text encoder pre-compute

In [8]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer, CLIPModel, CLIPVisionModelWithProjection

import json
import os
from imagenet_classes import IMAGENET2012_CLASSES

/home/choi/Downloads/miniconda3/envs/diff-rep/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
sd_path = "/scratch/choi/model/stable-diffusion-v1-5"
text_encoder = CLIPTextModel.from_pretrained(sd_path, subfolder="text_encoder")
tokenizer = CLIPTokenizer.from_pretrained(sd_path, subfolder="tokenizer")
# tokenizer2 = CLIPTokenizer.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")
# text_encoder = CLIPTextModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")
clip_vision_model = CLIPVisionModelWithProjection.from_pretrained(
    "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
)

In [10]:
type(clip_model.vision_model)

transformers.models.clip.modeling_clip.CLIPVisionTransformer

In [8]:
type(next(clip_model.parameters()))

torch.nn.parameter.Parameter

In [10]:
# print(clip_model.logit_scale.to("cuda:0"))
# clip_model.logit_scale.requires_grad_(True)
print(clip_model.logit_scale)
# clip_model.logit_scale.exp()
clip_model.logit_scale.item()

Parameter containing:
tensor(4.6052, requires_grad=True)


4.605170249938965

In [6]:
data = json.load(
    open("/scratch/choi/dataset/ImageNet100/_img_text_pair_train.json")
)  # list of dict: [{"image_file": "1.png", "text": "A dog"}]

In [7]:
text_list = []
label_list = os.listdir("/scratch/choi/dataset/ImageNet100/train")
for label in label_list:
    text_list.append("a photo of " + IMAGENET2012_CLASSES[label])
label_list.append("")
text_list.append("")

In [8]:
len(label_list)

101

In [9]:
text_tokens = tokenizer(
    text_list, padding="max_length", truncation=True, return_tensors='pt'
).input_ids

In [10]:
text_tokens.shape

torch.Size([101, 77])

In [15]:
device = 'cuda:2'
clip_model = clip_model.vision_model.to(device)
clip_vision_model = clip_vision_model.to(device)
# text_encoder = text_encoder.to(device)
# text_tokens = text_tokens.to(device)

In [3]:
import requests
from PIL import Image
from transformers import AutoProcessor

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = AutoProcessor.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

In [4]:
i = processor(images=image, return_tensors='pt')

In [14]:
i = i.to("cuda:0")
vis_model = clip_model.vision_model.to("cuda:0")
vis_proj = clip_model.visual_projection.to("cuda:0")
# clip_model = clip_model.to("cuda:1")
# clip_model.visual_projection(clip_model.vision_model(**i)[1])
tmp = vis_model(**i)
out = vis_proj(tmp[1])
print(out)
logit_scale = clip_model.logit_scale.to("cuda:0")
print(clip_model.logit_scale)
print(logit_scale)

tensor([[-0.7675, -0.3751,  0.4466,  ...,  0.2628,  0.6986, -0.3035]],
       device='cuda:0', grad_fn=<MmBackward0>)
Parameter containing:
tensor(4.6052, requires_grad=True)
tensor(4.6052, device='cuda:0', grad_fn=<ToCopyBackward0>)


In [6]:
print(out.shape)

torch.Size([1, 1024])


In [7]:
i = i.to('cuda:2')
clip_vision_model = clip_vision_model.to("cuda:2")
out2 = clip_vision_model(**i)

In [8]:
print(out2.keys())
print(out2[0])
print(out2[0].shape)

odict_keys(['image_embeds', 'last_hidden_state'])
tensor([[-0.7675, -0.3751,  0.4466,  ...,  0.2628,  0.6986, -0.3035]],
       device='cuda:2', grad_fn=<MmBackward0>)
torch.Size([1, 1024])


In [11]:
with torch.no_grad():
    out = text_encoder(text_tokens)[0]

In [12]:
out.shape

torch.Size([101, 77, 768])

In [13]:
a = dict(zip(label_list, out))

In [14]:
a['n02114855'].shape

torch.Size([77, 768])

In [16]:
torch.save(a, "IN100_text_embedding_dict_L.pt")

In [17]:
with torch.no_grad():
    out2 = clip_model.text_projection(clip_model.text_model(text_tokens)[1])

In [18]:
out2.shape

torch.Size([101, 1024])

In [19]:
b = dict(zip(label_list, out2))

In [20]:
b['n02114855'].shape

torch.Size([1024])

In [21]:
torch.save(b, 'IN100_text_embedding_dict_with_projection_H.pt')

In [ ]:
# TRY LOADING DICT
# -> NOTEBOOK TO SEPARATE FILE
# -> APPLY DICT TO CODE

In [66]:
import torch

In [67]:
a = torch.load("IN100_text_embedding_dict_L.pt")

In [68]:
a['n02109047'].shape

torch.Size([77, 768])

In [70]:
a[""]

tensor([[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
        [-0.3711, -1.4497, -0.3401,  ...,  0.9489,  0.1867, -1.1034],
        [-0.5107, -1.4629, -0.2926,  ...,  1.0419,  0.0701, -1.0284],
        ...,
        [ 0.5006, -0.9552, -0.6610,  ...,  1.6013, -1.0622, -0.2191],
        [ 0.4988, -0.9451, -0.6656,  ...,  1.6467, -1.0858, -0.2088],
        [ 0.4923, -0.8124, -0.4912,  ...,  1.6108, -1.0174, -0.2484]],
       device='cuda:1')

In [ ]:
a = torch.load("IN100_text_embedding_dict_L.pt")